In [20]:
pwd

'C:\\Users\\sebas'

In [30]:
from bs4 import BeautifulSoup
import re
import time
import requests

movie   = 'the_dark_knight'
name    = 'shu guan' # replace here with your name!
pageNum = 10

# create the movie url
url     = 'https://www.rottentomatoes.com/m/'+movie+'/reviews/'

# output file
with open('web analytics/'+name + '_' + movie+'.txt','w') as fw:
    
    # for each page out of the pageNum pages you want to parse
    for p in range(1,pageNum+1): 
        
        # tell user which page you're parsing
        print ('Getting page',p)
        
        # initialize html file to None
        html=None        
        
        # set URL to get appropriately
        #   if it is the first page
        if p==1: 
            # url for page 1
            pageLink=url 
        #   if it is tnot the first page
        else: 
            # url for other pages
            pageLink=url+'?page='+str(p)+'&sort=' # make the page url
            
        # try to scrape times
        for i in range(5): 
            try:
                # get url content
                response = requests.get(pageLink,headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36', })
                # get the html
                html=response.content
                # if we successuflly got the file, break the loop
                break 
            # requests.get() threw an exception, i.e., the attempt to get the response failed
            except:
                print ('failed attempt #',i)
                # wait 2 secs before trying again
                time.sleep(2)

        if not html:
            # couldnt get the page, ignore
            print('could not get page #', p)
            continue 
        
        # if we got the page, parse the html file
        # first, turn it into a beautiful soup object
        soup = BeautifulSoup(html.decode('ascii', 'ignore'),'lxml')
        
        # then get all the review divs
        reviews=soup.findAll('div', {'class':re.compile('review_table_row')})
        
        # grab the information for each review
        print ('Parsing page',p)
        
        
        for review in reviews:
            
            # initialize critic, rating, source, text, date
            critic,rating,source,text,date='NA','NA','NA','NA','NA'
            
            # 1. if there is critic name information, get it
            criticChunk=review.find('a',{'href':re.compile('/critic/')})
            if criticChunk: 
                critic=criticChunk.text.strip()
            
            # 2. if there is rating information, get it ;
            # The rating should be 'rotten' , 'fresh', or 'NA' if the review doesn't have a rating.
            ratingfresh=review.find("div", attrs={"class":"review_icon icon small fresh"} )
            ratingrotten=review.find("div", attrs={"class":"review_icon icon small rotten"} )
            if ratingfresh:
                rating='fresh'
            if ratingrotten:
                rating='rotten'
            
            
            # 3. if there is source information, get it
            sourceChunk=review.find("em", attrs={"class":"subtle critic-publication"})
            if sourceChunk:
                source=sourceChunk.text.strip()
            
            
            # 4. if there is text information, get it    
            
            T=review.find("div", attrs={"class":"the_review"})
            if T:
                text=T.text.strip()
            
            # 5. if there is date information, get it    
            D=review.find("div", attrs={"class":"review-date subtle small"})
            if D:
                date=D.text.strip()
            

                
            
            #write everything to file    		
            fw.write(critic+'\t'+rating+'\t'+source+'\t'+text+'\t'+date+'\n')

print ('Done!')

Getting page 1
Parsing page 1
Getting page 2
Parsing page 2
Getting page 3
Parsing page 3
Getting page 4
Parsing page 4
Getting page 5
Parsing page 5
Getting page 6
Parsing page 6
Getting page 7
Parsing page 7
Getting page 8
Parsing page 8
Getting page 9
Parsing page 9
Getting page 10
Parsing page 10
Done!
